In [5]:
import os
from dotenv import load_dotenv
import pandas as pd
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
import pymysql
print("✅ 라이브러리 임포트 완료!")

✅ 라이브러리 임포트 완료!


In [29]:
!pip install langchain-core

In [4]:
# API 키

# .env 파일에서 환경 변수를 불러옵니다.
load_dotenv()

# 환경 변수 읽기
openai_api_key = os.getenv("OPENAI_API_KEY")

# 파일 및 설정
CSV_FILE = "./seoul_restaurant_geo.csv"
QDRANT_URL = "http://localhost:6333"
COLLECTION_NAME = "seoul-restaurant"
                                                                                                                                                                                                              
print("✅ 설정 완료!")

✅ 설정 완료!


In [35]:
import pymysql

conn = pymysql.connect(
    host="127.0.0.1",
    port=3310,
    user="root",
    password="1234",
    db="kmedia_db",
    charset="utf8mb4",
    cursorclass=pymysql.cursors.DictCursor,
    autocommit=False,          # 조회만이면 True 편리, 쓰기 트랜잭션은 False 권장
    read_timeout=10,
    write_timeout=10,
    connect_timeout=5,
)

print("Test")

try:
    with conn.cursor() as cur:
        # 전송 구간 문자셋 고정
        cur.execute("SET NAMES utf8mb4;")

        sql = """
            SELECT id, location_name, city, drama_name, address, drama_desc, category
            FROM k_contents
            ORDER BY id DESC
            LIMIT 500
        """
        cur.execute(sql)               # ← 실행 누락 주의!
        rows = cur.fetchall()

        print("rows:", len(rows))
        if rows:
            print(rows[0])
finally:
    conn.close()

Test
rows: 500
{'id': 700, 'location_name': '봉은사', 'city': '서울', 'drama_name': '가족x멜로', 'address': '서울 강남구 봉은사로 531 봉은사역1번출구 도보5분', 'drama_desc': "[드라마 정보]  \n- 제목: 가족x멜로  \n- 주요 촬영지: 봉은사  \n- 주소: 서울 강남구 봉은사로 531 봉은사역1번출구 도보5분  \n- 카테고리: 사찰/종교  \n\n[장면 요약]  \n드라마 '가족x멜로'에서 봉은사는 주인공이 힘든 시기를 겪으며 고요한 마음을 찾기 위해 방문하는 장소로 등장합니다. 이곳에서 주인공은 고백의 장소에 서서 가족에 대한 그리움과 지난날의 추억을 회상하며 깊은 감정을 느끼게 됩니다. 어둠 속에서도 빛을 찾으려는 그 마음은, 봉은사의 고즈넉한 풍경과 어우러져 뭉클한 분위기를 연출합니다.  \n\n[여행 포인트]  \n봉은사를 방문하면 아름다운 절경을 감상하며 산책할 수 있습니다. 특히, 절 입구에서부터 시작되는 아기자기한 정원과 고요한 분위기 속에서 사진 촬영하기 좋은 스팟이 많습니다. 더불어, 인근에 위치한 카페나 식당에서 전통 차나 한식을 즐기며 여행의 여유를 만끽할 수 있습니다.  \n\n[감성 문장]  \n“봉은사의 고요한 정적 속에서, 마치 드라마 속 주인공이 된 듯한 특별한 순간을 만끽할 수 있습니다.”  ", 'category': '사찰/종교'}


In [30]:
rows[0]

{'id': 700,
 'location_name': '봉은사',
 'city': '서울',
 'drama_name': '가족x멜로',
 'address': '서울 강남구 봉은사로 531 봉은사역1번출구 도보5분',
 'drama_desc': "[드라마 정보]  \n- 제목: 가족x멜로  \n- 주요 촬영지: 봉은사  \n- 주소: 서울 강남구 봉은사로 531 봉은사역1번출구 도보5분  \n- 카테고리: 사찰/종교  \n\n[장면 요약]  \n드라마 '가족x멜로'에서 봉은사는 주인공이 힘든 시기를 겪으며 고요한 마음을 찾기 위해 방문하는 장소로 등장합니다. 이곳에서 주인공은 고백의 장소에 서서 가족에 대한 그리움과 지난날의 추억을 회상하며 깊은 감정을 느끼게 됩니다. 어둠 속에서도 빛을 찾으려는 그 마음은, 봉은사의 고즈넉한 풍경과 어우러져 뭉클한 분위기를 연출합니다.  \n\n[여행 포인트]  \n봉은사를 방문하면 아름다운 절경을 감상하며 산책할 수 있습니다. 특히, 절 입구에서부터 시작되는 아기자기한 정원과 고요한 분위기 속에서 사진 촬영하기 좋은 스팟이 많습니다. 더불어, 인근에 위치한 카페나 식당에서 전통 차나 한식을 즐기며 여행의 여유를 만끽할 수 있습니다.  \n\n[감성 문장]  \n“봉은사의 고요한 정적 속에서, 마치 드라마 속 주인공이 된 듯한 특별한 순간을 만끽할 수 있습니다.”  ",
 'category': '사찰/종교'}

In [36]:
df = pd.DataFrame(rows, columns=["id","location_name","city","drama_name","address","drama_desc","category"])
df[:1]

,id,location_name,city,drama_name,address,drama_desc,category
0,700,봉은사,서울,가족x멜로,서울 강남구 봉은사로 531 봉은사역1번출구 도보5분,[드라마 정보] \n- 제목: 가족x멜로 \n- 주요 촬영지: 봉은사 \n- ...,사찰/종교


# k-contents 벡터화

In [37]:
# 셀 3: CSV 로드 및 전처리 (핵심!)
print("📂 CSV 파일 로딩 및 전처리 중...")

df = pd.DataFrame(rows)

print(f"✅ 총 {len(df)}개 레스토랑 로드")
print(f"\n컬럼: {df.columns.tolist()}")

# 의미 있는 Document 생성
docs = []
for idx, row in df.iterrows():
    # 중요한 정보만 조합
    text = f"""
촬영지명: {row['location_name']}
위치: {row['address']}
드라마명: {row['drama_name']}
도시: {row['city']}
드라마설명: {row['drama_desc']}
카테고리: {row['category']}
    """.strip()
    
    # Document 생성
    doc = Document(
        page_content=text,
        metadata={
            'location_name': row['location_name'],
            'address': row['address'],
            'drama_name': row['drama_name'],
            'city': row['city'],
            'drama_desc': row['drama_desc'],
            'category': row['category']
        }
    )
    docs.append(doc)

print(f"✅ {len(docs)}개 Document 생성 완료!")
print(f"\n첫 번째 장소 샘플:")
print(docs[0].page_content)

📂 CSV 파일 로딩 및 전처리 중...
✅ 총 500개 레스토랑 로드

컬럼: ['id', 'location_name', 'city', 'drama_name', 'address', 'drama_desc', 'category']
✅ 500개 Document 생성 완료!

첫 번째 장소 샘플:
촬영지명: 봉은사
위치: 서울 강남구 봉은사로 531 봉은사역1번출구 도보5분
드라마명: 가족x멜로
도시: 서울
드라마설명: [드라마 정보]  
- 제목: 가족x멜로  
- 주요 촬영지: 봉은사  
- 주소: 서울 강남구 봉은사로 531 봉은사역1번출구 도보5분  
- 카테고리: 사찰/종교  

[장면 요약]  
드라마 '가족x멜로'에서 봉은사는 주인공이 힘든 시기를 겪으며 고요한 마음을 찾기 위해 방문하는 장소로 등장합니다. 이곳에서 주인공은 고백의 장소에 서서 가족에 대한 그리움과 지난날의 추억을 회상하며 깊은 감정을 느끼게 됩니다. 어둠 속에서도 빛을 찾으려는 그 마음은, 봉은사의 고즈넉한 풍경과 어우러져 뭉클한 분위기를 연출합니다.  

[여행 포인트]  
봉은사를 방문하면 아름다운 절경을 감상하며 산책할 수 있습니다. 특히, 절 입구에서부터 시작되는 아기자기한 정원과 고요한 분위기 속에서 사진 촬영하기 좋은 스팟이 많습니다. 더불어, 인근에 위치한 카페나 식당에서 전통 차나 한식을 즐기며 여행의 여유를 만끽할 수 있습니다.  

[감성 문장]  
“봉은사의 고요한 정적 속에서, 마치 드라마 속 주인공이 된 듯한 특별한 순간을 만끽할 수 있습니다.”  
카테고리: 사찰/종교


In [38]:
print("🤖 OpenAI 임베딩 모델 준비 중...")

embedding = OpenAIEmbeddings(model="text-embedding-ada-002")

print("✅ 임베딩 모델 준비 완료!")

🤖 OpenAI 임베딩 모델 준비 중...
✅ 임베딩 모델 준비 완료!


In [39]:
# 셀 5: Qdrant 연결 및 컬렉션 생성
print("🔌 Qdrant 연결 중...")

qdrant_client = QdrantClient(url=QDRANT_URL)

# 기존 컬렉션 삭제
try:
    qdrant_client.delete_collection(collection_name=COLLECTION_NAME)
    print("🗑️  기존 컬렉션 삭제됨")
except:
    print("ℹ️  기존 컬렉션 없음")

# 새 컬렉션 생성
qdrant_client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE)
)

print(f"✅ 컬렉션 '{COLLECTION_NAME}' 생성 완료!")

🔌 Qdrant 연결 중...
ℹ️  기존 컬렉션 없음


ResponseHandlingException: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다

In [22]:
# 셀 6: 데이터 임베딩 및 저장
print(f"\n💾 {len(docs)}개 레스토랑 임베딩 생성 및 저장 중...")
print("⏳ OpenAI API 호출 중... (약 10-15분 소요)")

vector_store = Qdrant.from_documents(
    documents=docs,
    embedding=embedding,
    url=QDRANT_URL,
    collection_name=COLLECTION_NAME,
)

print(f"\n✅ 총 {len(docs)}개 레스토랑 저장 완료!")
print(f"📊 Qdrant UI: {QDRANT_URL}/dashboard")


💾 1178개 레스토랑 임베딩 생성 및 저장 중...
⏳ OpenAI API 호출 중... (약 10-15분 소요)

✅ 총 1178개 레스토랑 저장 완료!
📊 Qdrant UI: http://localhost:6333/dashboard


In [23]:
# 셀 7: 저장 확인
collection_info = qdrant_client.get_collection(COLLECTION_NAME)

print(f"📊 컬렉션 정보:")
print(f"  - 컬렉션명: {COLLECTION_NAME}")
print(f"  - 저장된 레코드: {collection_info.points_count}개")
print(f"  - 벡터 차원: {collection_info.config.params.vectors.size}")

📊 컬렉션 정보:
  - 컬렉션명: seoul-restaurant
  - 저장된 레코드: 1178개
  - 벡터 차원: 1536


In [24]:
# 셀 7: 저장 확인
collection_info = qdrant_client.get_collection(COLLECTION_NAME)

print(f"📊 컬렉션 정보:")
print(f"  - 컬렉션명: {COLLECTION_NAME}")
print(f"  - 저장된 레코드: {collection_info.points_count}개")
print(f"  - 벡터 차원: {collection_info.config.params.vectors.size}")

📊 컬렉션 정보:
  - 컬렉션명: seoul-restaurant
  - 저장된 레코드: 1178개
  - 벡터 차원: 1536


In [25]:
# 셀 8: 검색 함수 정의
def search_restaurant(query, k=5):
    """레스토랑 검색"""
    print(f"🔍 질의: '{query}'")
    print("=" * 60)
    
    results = vector_store.similarity_search(query, k=k)
    
    for i, doc in enumerate(results, 1):
        print(f"\n[{i}위]")
        print("-" * 60)
        
        # 메타데이터 출력
        meta = doc.metadata
        print(f"🍽️  이름: {meta.get('name', 'N/A')}")
        print(f"📍 위치: {meta.get('place', 'N/A')}")
        print(f"🏷️  종류: {meta.get('type', 'N/A')}")
        print(f"💰 가격: {meta.get('price', 'N/A')}")
        print(f"📞 전화: {meta.get('tel', 'N/A')}")
        
        print(f"\n내용:")
        print(doc.page_content[:200] + "...")
        print("-" * 60)
    
    return results

print("✅ 검색 함수 준비 완료!")

✅ 검색 함수 준비 완료!


In [31]:
# 셀 9: 검색 테스트 1 - 지역
search_restaurant("피자", k=3)

🔍 질의: '피자'

[1위]
------------------------------------------------------------
🍽️  이름: 지노스 뉴욕 피자
📍 위치: 04345  서울 용산구 녹사평대로40길 46 (이태원동)  2층
🏷️  종류: 음식점
💰 가격: None
📞 전화: 0507-1403-2234

내용:
레스토랑: 지노스 뉴욕 피자
위치: 04345  서울 용산구 녹사평대로40길 46 (이태원동)  2층
설명: 도끼를 단골손님에서 사장으로 거듭나게 한 피자 맛집.

모든 피자는 가공하지 않은 원재료를 빚어 만듭니다. 간이 강한 오리지널 피자부터 한국인의 입맛에 고안된 담백한 피자까지 다양한 종류의 피자를 즐길 수 있습니다. 메인 메뉴는 토마토소스 베이스의 ...
------------------------------------------------------------

[2위]
------------------------------------------------------------
🍽️  이름: 대장장이화덕피자집
📍 위치: 03056  서울 종로구 북촌로 42-4 (가회동)
🏷️  종류: 음식점
💰 가격: None
📞 전화: 0507-1315-4298

내용:
레스토랑: 대장장이화덕피자집
위치: 03056  서울 종로구 북촌로 42-4 (가회동)
설명: 화덕에 구운 담백한 피자 맛으로 유명세를 타는 곳.

화덕에 구운 담백한 피자 맛으로 유명세를 타는 곳. 좋은 재료를 아낌없이 듬뿍 담은 피자를 맛볼 수 있습니다. 주말과 공휴일은 하루 전에 예약해야 합니다.
대표메뉴: 마르게리따, 고르곤졸라, 루꼴라 피자, 감베...
------------------------------------------------------------

[3위]
------------------------------------------------------------
🍽️  이름: PB플러스
📍 위치: 04406  서울 용산구 우사단로 19-1  1

[Document(metadata={'restaurant_id': 321, 'name': '지노스 뉴욕 피자', 'place': '04345\xa0\xa0서울 용산구 녹사평대로40길 46 (이태원동)\xa0\xa02층', 'latitude': 37.5343987, 'longitude': 126.988211, 'type': '음식점', 'price': None, 'tel': '0507-1403-2234', 'website': ' https://www.instagram.com/ginospizza/', '_id': 'b135bb62-c84a-44df-bcea-73b2e9fde3cc', '_collection_name': 'seoul-restaurant'}, page_content='레스토랑: 지노스 뉴욕 피자\n위치: 04345\xa0\xa0서울 용산구 녹사평대로40길 46 (이태원동)\xa0\xa02층\n설명: 도끼를 단골손님에서 사장으로 거듭나게 한 피자 맛집.\n\n모든 피자는 가공하지 않은 원재료를 빚어 만듭니다. 간이 강한 오리지널 피자부터 한국인의 입맛에 고안된 담백한 피자까지 다양한 종류의 피자를 즐길 수 있습니다. 메인 메뉴는 토마토소스 베이스의 ‘브룩클린스 베스트’와 알프레도 소스 베이스의 ‘트러플 슈룸’. 트러플 오일을 넣어 담백하고 풍미가 깊은 ‘트러플 슈룸’은 도끼를 단골손님에서 사장으로 거듭나게 한 이유이기도 합니다. 지노스 피자는 격식 없이 즐길 수 있는 피자가 생각날때 언제든 편하게 찾을 수 있는 식당입니다.\n대표메뉴: 브루클린스베스트, 시금치 알프레도, 뉴욕슈프림\n음식종류: 음식점\n가격대: nan\n근처역: 지하철 6호선 녹사평역 1번 출구, 236m\n영업시간: 평일 11:30 - 15:00 / 17:00 - 22:00\n토요일 11:30 - 22:00 / 일요일 11:30 - 21:00\n전화: 0507-1403-2234'),
 Document(metadata={'restaurant_id': 239, 'name': '대장장이화덕

In [41]:
# 셀 9: 검색 테스트 1 - 지역
search_restaurant("설렁탕 맛집", k=5)

🔍 질의: '설렁탕 맛집'

[1위]
------------------------------------------------------------
🍽️  이름: 정애맛담
📍 위치: 06997  서울 동작구 동작대로29나길 10
🏷️  종류: 카페, 디저트
💰 가격: 1만원대
📞 전화: 0507-1355-9096

내용:
레스토랑: 정애맛담
위치: 06997  서울 동작구 동작대로29나길 10
설명: 서울 남성시장에 위치한 정애맛담은 한국의 전통음식인 떡에 현대적인 감각을 더한 퓨전 떡집.

서울 남성시장에 위치한 정애맛담은 한국의 전통음식인 떡에 현대적인 감각을 더한 퓨전 떡집입니다. 고물을 묻혀 먹는 말랑 인절미와 빵 대신 담백한 설기에 팥앙금과 버터를 넣은 앙버떡이 인...
------------------------------------------------------------

[2위]
------------------------------------------------------------
🍽️  이름: 영동설렁탕
📍 위치: 06525  서울 서초구 강남대로101안길 24 (잠원동)
🏷️  종류: 음식점
💰 가격: 1만원대
📞 전화: 02-543-4716

내용:
레스토랑: 영동설렁탕
위치: 06525  서울 서초구 강남대로101안길 24 (잠원동)
설명: 30년 전통 손맛을 자랑하는 집. 설렁탕 한 그릇으로 승부하는 곳으로, 설렁탕 외의 메뉴는 수육 한 가지. 기름기가 싫거나 다이어트 중인 사람은 주문할 때 기름빼기 설렁탕을 주문.

30년 전통 손맛을 자랑하는 집. 설렁탕 한 그릇으로 승부하는 곳으로, 설렁탕 외...
------------------------------------------------------------

[3위]
------------------------------------------------------------
🍽️  이름: 피맛골
📍 위치: 03155  서울 종로구 종로3길 17
🏷️  종류: 음식점
💰 가격:

[Document(metadata={'restaurant_id': 718, 'name': '정애맛담', 'place': '06997\xa0\xa0서울 동작구 동작대로29나길 10', 'latitude': 37.4890369, 'longitude': 126.9806053, 'type': '카페, 디저트', 'price': '1만원대', 'tel': '0507-1355-9096', 'website': ' https://www.jeongaematdam.com/', '_id': '76e71e6c-f2dc-4897-8454-defe46eefd0e', '_collection_name': 'seoul-restaurant'}, page_content='레스토랑: 정애맛담\n위치: 06997\xa0\xa0서울 동작구 동작대로29나길 10\n설명: 서울 남성시장에 위치한 정애맛담은 한국의 전통음식인 떡에 현대적인 감각을 더한 퓨전 떡집.\n\n서울 남성시장에 위치한 정애맛담은 한국의 전통음식인 떡에 현대적인 감각을 더한 퓨전 떡집입니다. 고물을 묻혀 먹는 말랑 인절미와 빵 대신 담백한 설기에 팥앙금과 버터를 넣은 앙버떡이 인기 메뉴입니. 이외에도 매일 아침 좋은 재료로 만드는 수많은 종류의 신선한 떡을 생산하여 판매하기 때문에 다양하게 맛볼 수 있습니다.\n대표메뉴: 말랑인절미, 앙버떡\n음식종류: 카페, 디저트\n가격대: 1만원대\n근처역: 지하철 4 / 7호선 총신대입구(이수)역 14번 출구, 197m\n영업시간: 이용시간: 06:00 - 19:30 또는 20:00\n전화: 0507-1355-9096'),
 Document(metadata={'restaurant_id': 286, 'name': '영동설렁탕', 'place': '06525\xa0\xa0서울 서초구 강남대로101안길 24 (잠원동)', 'latitude': 37.51615299, 'longitude': 127.0173661, 'type': '음식점', 'price': '1만원대', 'tel': '02-543-4

In [25]:
search_restaurant("한식당 추천", k=5)

🔍 질의: '한식당 추천'

[1위]
------------------------------------------------------------
﻿restaurant_id: 165
name: 남도식당
place: 03156  서울 종로구 종로3길 42
image_path: images/남도식당_165.jpg
description: 저렴한 가격에 푸짐하게 한 상 차려진 식사를 할 수 있는 곳. 몇십 년 전통의 고추장 양념 돼지고기 맛이 일품.

저렴한 가격에 푸짐하게 한 상 차려진 식사를 할 수 있는 곳. 몇십 년 전통의 고추장 양념 돼지고기 맛이 일품입니다. 식사 시간이 되면 연탄불 석쇠에 구운 고기를 먹기 위해 찾아오는 손님들로 붐비는 곳 중 하나입니다. 한정식 외에도 혼
------------------------------------------------------------

[2위]
------------------------------------------------------------
﻿restaurant_id: 611
name: 남한강 매운탕
place: 05295  서울 강동구 양재대로 1594 (명일동)
image_path: images/남한강_매운탕_611.jpg
description: 경기도 양평에서 양식한 메기, 강원도 양구군의 자연산 민물고기로 만든 매운탕집.

어부가 직접 잡은 메기, 쏘가리, 빠가사리, 민물참게로 도심에서 즐기는 칼칼하고 시원한 민물매운탕

 

강동구 맛집으로 소문난 남한강 매운탕은 경기도 양평에서 양식한 메기 및 강원도 양구군에서 어부가 직접 잡은 자연산 민물고기와 및 민물참게
------------------------------------------------------------

[3위]
------------------------------------------------------------
﻿restaurant_id: 447
name: 안동장
place: 04550  서울특별시 중구 을지로 124

[Document(metadata={'source': './seoul_restaurant_geo.csv', 'row': 160, '_id': 'bf6bc23a-a54b-45a7-a0c7-67bdd0217249', '_collection_name': 'seoul-restaurant'}, page_content='\ufeffrestaurant_id: 165\nname: 남도식당\nplace: 03156\xa0\xa0서울 종로구 종로3길 42\nimage_path: images/남도식당_165.jpg\ndescription: 저렴한 가격에 푸짐하게 한 상 차려진 식사를 할 수 있는 곳. 몇십 년 전통의 고추장 양념 돼지고기 맛이 일품.\n\n저렴한 가격에 푸짐하게 한 상 차려진 식사를 할 수 있는 곳. 몇십 년 전통의 고추장 양념 돼지고기 맛이 일품입니다. 식사 시간이 되면 연탄불 석쇠에 구운 고기를 먹기 위해 찾아오는 손님들로 붐비는 곳 중 하나입니다. 한정식 외에도 혼자 온 고객들을 위해, 현미 백반이라는 상차림을 따로 마련해 두고 있습니다.\nLatitude: 37.5713444\nLongitude: 126.9793117\nprice: \nnearsubway: 지하철 5호선 광화문역 2번 출구, 253m\nsignaturemenu: 연탄삼겹살, 연탄소불고기, 육회, 병어찜, 김치전골\noperatingday: 월요일~토요일\nwebsite: \ntype: 음식점\nnotice: \noperatinghour: 10:00 ~ 14:00 / 16:00 ~ 22:00\ntel: 02-734-0719\nholidays: 매주 일요일, 명절 휴무'),
 Document(metadata={'source': './seoul_restaurant_geo.csv', 'row': 602, '_id': '34665e2b-9934-4efc-b96b-0024234d232f', '_collection_name': 'seoul-restaurant'}, page_content='\ufeffr

In [26]:
# 먼저 CSV 구조 확인
import pandas as pd

df = pd.read_csv("./seoul_restaurant_geo.csv")

print("📊 CSV 기본 정보:")
print(f"총 행 수: {len(df)}")
print(f"\n컬럼 목록:")
print(df.columns.tolist())
print(f"\n첫 3개 데이터:")
print(df.head(3))
print(f"\n데이터 샘플:")
print(df.iloc[0].to_dict())

📊 CSV 기본 정보:
총 행 수: 1178

컬럼 목록:
['restaurant_id', 'name', 'place', 'image_path', 'description', 'Latitude', 'Longitude', 'price', 'nearsubway', 'signaturemenu', 'operatingday', 'website', 'type', 'notice', 'operatinghour', 'tel', 'holidays']

첫 3개 데이터:
   restaurant_id        name                            place  \
0              1        통인시장         03036  서울 종로구 자하문로15길 18   
1              2         삼청각  02822  서울 성북구 대사관로 3 (성북동, 삼청각)   
2              3  카몽(KAMONG)   06647  서울 서초구 서초대로42길 17 (서초동)   

                image_path                                        description  \
0        images/통인시장_1.jpg  SBS [런닝맨], KBS [1박2일]의 촬영지이자, 엽전 도시락과 기름 떡볶이로 ...   
1         images/삼청각_2.png  삼청각은 전통예술복합문화공간. 한국의 유명한 대목장 박광석과 대목수 조승원이 참가해...   
2  images/카몽(KAMONG)_3.jpg  엑소의 멤버 ‘카이가 꿈을 꾸다’ 라는 뜻을 가진 카페 카몽은 카이의 누나가 운영하...   

    Latitude   Longitude price                                nearsubway  \
0  37.580755  126.969996   NaN                  지하철 3호선 경복궁역 2번 출구, 515m   
1  37.59

In [42]:
# 이 부분이 제대로 실행되었나요?
try:
    qdrant_client.delete_collection(collection_name="seoul-restaurant")
    print("🗑️  기존 컬렉션 삭제됨")  # ← 이게 출력되었나요?
except:
    print("ℹ️  기존 컬렉션 없음")

🗑️  기존 컬렉션 삭제됨


# 유저데이터 벡터화

# 장소데이터 벡터화 

# 북마크 데이터 벡터화 